In [1]:
import geopandas

In [2]:
df = geopandas.read_file("global_tectonics/plates&provinces/boundaries.shp")
#df = geopandas.read_file("tectonicplates/PB2002_boundaries.shp")

In [3]:
geometries = df["geometry"]

In [4]:
simple_geom = [geom for geom in geometries if geom.geom_type == "LineString"]
multi_geom = [geom for geom in geometries if geom.geom_type != "LineString"]

In [5]:
combine = []

for mgeom in multi_geom:
    combine += list(mgeom.geoms)  

In [6]:
geometries = simple_geom + combine

In [7]:
from pyproj import Proj
to_stereographic = Proj("+proj=stere +lat_0=90 +R=1.0 +lat_ts=0")

## GMSH

In [8]:
import gmsh

In [9]:
gmsh.initialize()

In [10]:
gmsh.model.add("Sphere")

In [11]:
lc = 0.1

pt1 = gmsh.model.geo.add_point(0,0,0, meshSize=lc)
pt2 = gmsh.model.geo.add_point(1,0,0, meshSize=lc)
pt3 = gmsh.model.geo.add_point(-1,0,0, meshSize=lc)
pt4 = gmsh.model.geo.add_point(0,-1,0, meshSize=lc)
pt5 = gmsh.model.geo.add_point(0,1,0, meshSize=lc)
pt6 = gmsh.model.geo.add_point(0,0,1, meshSize=lc)
pt7 = gmsh.model.geo.add_point(0,0,-1, meshSize=lc)

In [12]:
circle1 = gmsh.model.geo.add_circle_arc(2, 1, 4)
circle2 = gmsh.model.geo.add_circle_arc(4, 1, 3)
circle3 = gmsh.model.geo.add_circle_arc(3, 1, 5)
circle4 = gmsh.model.geo.add_circle_arc(5, 1, 2)
circle5 = gmsh.model.geo.add_circle_arc(2, 1, 6)
circle6 = gmsh.model.geo.add_circle_arc(6, 1, 3)
circle7 = gmsh.model.geo.add_circle_arc(3, 1, 7)
circle8 = gmsh.model.geo.add_circle_arc(7, 1, 2)
circle9 = gmsh.model.geo.add_circle_arc(4, 1, 6)
circle10 = gmsh.model.geo.add_circle_arc(6, 1, 5)
circle11 = gmsh.model.geo.add_circle_arc(5, 1, 7)
circle12 = gmsh.model.geo.add_circle_arc(7, 1, 4)

In [13]:
loop14 = gmsh.model.geo.add_curve_loop([2, 7, 12])
surf1 = gmsh.model.geo.add_surface_filling([loop14])
loop16 = gmsh.model.geo.add_curve_loop([2, -6, -9])
surf2 = gmsh.model.geo.add_surface_filling([loop16])
loop18 = gmsh.model.geo.add_curve_loop([3, -10, 6])
surf3 = gmsh.model.geo.add_surface_filling([loop18])
loop20 = gmsh.model.geo.add_curve_loop([3, 11, -7])
surf4 = gmsh.model.geo.add_surface_filling([loop20])
loop22 = gmsh.model.geo.add_curve_loop([4, -8, -11])
surf5 = gmsh.model.geo.add_surface_filling([loop22])
loop24 = gmsh.model.geo.add_curve_loop([4, 5, 10])
surf6 = gmsh.model.geo.add_surface_filling([loop24])
loop26 = gmsh.model.geo.add_curve_loop([1, 9, -5])
surf7 = gmsh.model.geo.add_surface_filling([loop26])
loop28 = gmsh.model.geo.add_curve_loop([1, -12, 8])
surf8 = gmsh.model.geo.add_surface_filling([loop28])

In [14]:
sphere = gmsh.model.geo.add_geometry('PolarSphere', [0., 0., 0., 1.0])

In [15]:
import numpy as np

def create_surface_from_geometry(geometry):

    coords = list(geometry.coords)
    coords_lon_lat_degrees = np.array(coords)
    
    x, y = to_stereographic(coords_lon_lat_degrees[:,0], 
                            coords_lon_lat_degrees[:,1])
    
    coords_stereographic = np.zeros_like(coords)
    coords_stereographic[:, 0] = x
    coords_stereographic[:, 1] = y
    
    ## Add points
    
    point_tags = []

    # Here we account for the fact that, in a polygon shapefile, the first and
    # last node of a polygon are the same.
    for index in range(len(coords_stereographic) - 1):
        x = coords_stereographic[index, 0]
        y = coords_stereographic[index, 1]
        tag = gmsh.model.geo.add_point_on_geometry(sphere, x, y)
        point_tags.append(tag)
        
    point_tags = point_tags + [point_tags[0]]
    
    ## Add lines
    
    line_tags = []

    for index in range(len(point_tags) - 1):
        pt1, pt2 = point_tags[index], point_tags[index + 1]
        tag = gmsh.model.geo.add_line(pt1, pt2)
        line_tags.append(tag)
    
    cl1 = gmsh.model.geo.add_curve_loop(line_tags)
    return line_tags
    #gmsh.model.geo.add_plane_surface([cl1])

In [16]:
line_tags = []

for geometry in geometries:
    line_tags += create_surface_from_geometry(geometry)

In [17]:
gmsh.model.geo.synchronize()

In [18]:
gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "CurvesList", line_tags)


gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "InField", 1)
gmsh.model.mesh.field.setNumber(2, "SizeMin", 0.01)
gmsh.model.mesh.field.setNumber(2, "SizeMax", 0.1)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.01)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.2)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

In [19]:
gmsh.write("Model.vtk")

Info    : Writing 'Model.vtk'...
Info    : Done writing 'Model.vtk'


In [20]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.10.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220501
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
